# Módulo de Preprocesamiento II (Ejercicios Prácticos)

> Ejemplos generales de preprocesamiento de datos en python



# **Importar Dataset**

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Importar y Exportar datos con PANDAS

In [6]:
import pandas as pd # Importar la librería Pandas
import numpy as np  # Importar la librería Numpy

In [ ]:
df = pd.read_csv("glass_with_nulls.csv", sep=",") # Importar del archivo de datos y guardarlo en un DATAFRAME
df.head(4)                    # Muestra los cuatro pimeros registros


#  **Variables Categóricas a Numéricas** 

OrdinalEncoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()          #Codificar consecutivamnete cada valor nominal
#sklearn.preprocessing.LabelEncoder para la clase
#enc = OrdinalEncoder()
#enc.fit(df[["Sex","Blood", "Study"]])
#df[["Sex","Blood", "Study"]] = enc.transform(df[["Sex","Blood", "Study"]])
#label_encoder = preprocessing.LabelEncoder()
#dataset_encoded["salary"] = label_encoder.fit_transform(dataset_encoded["salary"])

Type_array = np.array(df['Type']).reshape(-1,1) 
Type_transformado = ordinal_encoder.fit_transform(Type_array)
df_transformado_OE=df.copy()
df_transformado_OE.insert(10,"Type_Encoded",Type_transformado)   #insertar una columna en un dataframe
df_transformado_OE.head(4)          #El problema es que los tipos de vidrio no tienen significado numérico al medir distancias

OneHotEncoder

In [10]:
#Codificar cada valor nominal en una nueva columna (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

one_hot_columns = pd.get_dummies(df.Type,dtype=None)


In [11]:
df_transformado_OHE=df.copy()
df_transformado_OHE.drop(['Type'], axis=1, inplace=True) 
df_transformado_OHE=pd.concat([df_transformado_OHE, one_hot_columns],axis=1)

In [ ]:
df_transformado_OHE

#  **Valores Nulos** 

Técnica Estadística 

In [13]:
import numpy as np
from sklearn.impute import SimpleImputer   #(https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [ ]:
# Extraer y transformar el atributo Na
Na_array = np.array(df['Na']).reshape(-1,1) 
print(Na_array)

In [14]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') 
Na_imputed = imputer.fit_transform(Na_array)

In [ ]:
Na_imputed

Técnica de DataMining para imputar valores

In [16]:
from sklearn.impute import KNNImputer

In [17]:
imputer = KNNImputer()    #Se desea remplazar desconocidos por los registros más similares
                          #Este método no permite comparar cadenas!
#imputer.fit_transform(df)   #con cadenas no es posible con lo cual se debe utilizar OHE


In [ ]:
df_transformado_OHE_Sin_Nulos = pd.DataFrame(imputer.fit_transform(df_transformado_OHE),columns = df_transformado_OHE.columns)
df_transformado_OHE_Sin_Nulos

In [ ]:
df_transformado_OHE_Sin_Nulos.isna().any()

# **Valores Atípicos**

In [ ]:
!pip install pyod

from pyod.models.lof import LOF  #https://pyod.readthedocs.io/en/latest/_modules/pyod/models/lof.html
from pyod.models.knn import KNN  #https://pyod.readthedocs.io/en/latest/_modules/pyod/models/knn.html

In [ ]:
imputer1 = LOF() 
Data_Numpy_Array = df_transformado_OHE_Sin_Nulos.to_numpy()  
imputer1.fit(Data_Numpy_Array)                         # Todo el dataset
pred = imputer1.predict(Data_Numpy_Array)              # outlier labels (0 or 1)
scores = imputer1.decision_function(Data_Numpy_Array)  # outlier scores
print (pred)
print (scores)

In [ ]:
imputer2 = KNN() 
imputer2.fit(Na_imputed)                         # Se podría utilizar para un unico atributo pero si fuese el caso es recomendable técnicas estadísticas
pred = imputer2.predict(Na_imputed)              # outlier labels (0 or 1)
scores = imputer2.decision_function(Na_imputed)  # outlier scores
print (pred)
print (scores)

#Selección de Características

Método Wrapper

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv("glass.csv", sep=",") # Importar del archivo de datos y guardarlo en un DATAFRAME
df_norm = MinMaxScaler().fit_transform(np.array(df.drop('Type', axis=1))) 
y = np.array(df['Type'])

model = LogisticRegression(solver='lbfgs', multi_class='auto')
rfe_selector = RFE(model, n_features_to_select=3)
fit = rfe_selector.fit(df_norm, y)

print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Método Embebido

In [25]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# feature extraction
model = ExtraTreesClassifier(n_estimators=10)
model.fit(df_norm, y)
print(model.feature_importances_)   # a mayor score mayor importancia 